In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density'

In [4]:
url_pop_html = requests.get(url)

In [5]:
url_pop_bs = bs(url_pop_html.text)

In [7]:
tables = url_pop_bs.find_all('table',class_='wikitable')

In [16]:
pop_table = tables[0]

In [24]:
fields = [th.text.strip() for th in pop_table.find_all('tr')[0].find_all('th')]

In [26]:
fields

['Rank',
 'Country (or dependent territory)',
 'Area',
 'Population',
 'Density',
 'Date',
 'Population source']

In [42]:
trs = pop_table.find_all('tr')[2:-2]

In [43]:
rows = []

In [44]:
for tr in trs:
    tds = tr.find_all('td')
    if not tds:
        continue
    data = [td.text.strip() for td in tds]
    del data[2]
    del data[4]
    rows.append({key:value for key,value in zip(fields,data)})

In [45]:
pop_df = pd.DataFrame(rows)

In [46]:
pop_df

,Rank,Country (or dependent territory),Area,Population,Density,Date,Population source
0,–,Macau (China),45,"696,100","54,799","September 30, 2019",Official quarterly estimate
1,1,Monaco,0.78,"38,300","49,107","December 31, 2018",Official estimate
2,2,Singapore,279,"5,703,600","20,446","July 1, 2019",Official estimate
3,–,Hong Kong (China),427,"7,500,700","17,565","December 31, 2019",Official estimate
4,–,Gibraltar (United Kingdom),2.6,"33,701","12,836","July 1, 2019",UN projection
...,...,...,...,...,...,...,...
248,194,Mongolia,"603,902","3,238,479",5.4,"December 31, 2018",Official estimate
249,–,Pitcairn Islands (United Kingdom),18,56,3.1,"September 20, 2016",2013 census result
250,–,Falkland Islands (United Kingdom),"4,700","2,563",0.54,"April 15, 2012",2012 census result
251,–,Svalbard and Jan Mayen (Norway),"23,706","2,655",0.10,"September 1, 2012",Official estimate
